In [ ]:
'''
1. データセットとデータローダーを用意
'''
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# MNISTデータセットの訓練データを用意
dataset = datasets.MNIST(
    # mnistフォルダーに保存
    # パスは環境に合わせて書き換えることが必要
    root='/content/drive/MyDrive/Colab Notebooks/GAN/DCGAN_PyTorch/mnist',
    download=True,
    train=True,
    # トランスフォームオブジェクトを設定
    transform=transforms.Compose(
        # Tensorオブジェクトに変換
        [transforms.ToTensor(),
         # データを平均0.5、標準偏差0.5の標準正規分布で正規化
         # チャネル数は1なのでタプルの要素も1
         transforms.Normalize((0.5,), (0.5,))]
         )
    )

# ミニバッチのサイズ
batch_size=50

# 訓練データをセットしたデータローダーを作成する
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size, # ミニバッチのサイズは50
    shuffle=True,          # データをシャッフルしてから抽出
    )

# 使用可能なデバイスを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


device: cuda:0


In [ ]:
'''
2. 識別器のクラスを定義 
'''
import torch.nn as nn

class Discriminator(nn.Module):
    '''識別器のクラス

    Attributes:
      layers: Sequentialオブジェクトのリスト
    '''
    def __init__(self):
        '''識別器のネットワークを構築する
        '''
        super(Discriminator, self).__init__()

        in_ch = 1      # 入力画像のチャネル数
        start_ch = 128 # 先頭層の出力チャネル数

        # 識別器のネットワークを定義する
        self.layers = nn.ModuleList([
            # 第1層: (bs, 1, 28, 28) -> (bs, 128, 14, 14)
            nn.Sequential(
                # 畳み込み
                nn.Conv2d(in_ch,    # 入力のチャネル数は1
                          start_ch, # フィルター数は128
                          4,  # 4×4のフィルター
                          2,  # ストライドは2
                          1), # 上下左右にサイズ1のパディング
                # LeakyReLU関数を適用
                # 論文に従って負の勾配を制御する係数を
                # 0.2(デフォルトは0.01)に設定
                nn.LeakyReLU(negative_slope=0.2)
            ), 
            # 第2層: (bs, 128, 14, 14) -> (bs, 256, 7, 7)
            nn.Sequential(
                # 畳み込み
                nn.Conv2d(start_ch,     # 入力のチャネル数は128
                          start_ch * 2, # フィルター数は128×2
                          4,  # 4×4のフィルター
                          2,  # ストライドは2
                          1), # 上下左右にサイズ1のパディング
                # 出力値を正規化する(チャネル数は128×2)
                nn.BatchNorm2d(start_ch * 2),
                # LeakyReLU関数を適用
                nn.LeakyReLU(negative_slope=0.2)
            ),
            # 第3層: (bs, 256, 7, 7) -> (bs, 512, 3, 3)
            nn.Sequential(
                # 畳み込み
                nn.Conv2d(start_ch * 2, # 入力のチャネル数は128×2
                          start_ch * 4, # フィルター数は128×4
                          3,  # 3×3のフィルター
                          2,  # ストライドは2
                          0), # パディングは0(なし)
                # 出力値を正規化する(チャネル数は128×4)
                nn.BatchNorm2d(start_ch * 4),
                # leaky ReLU関数を適用
                nn.LeakyReLU(negative_slope=0.2)
            ),
            # 第4層: (bs, 512, 3, 3) -> (bs, 1, 1, 1)
            nn.Sequential(
                nn.Conv2d(start_ch * 4, # 入力のチャネル数は128×4
                          1,  # フィルター数は1
                          3,  # 3×3のフィルター
                          1,  # ストライドは1
                          0), # パディングは0(なし)
                # 最終出力にはシグモイド関数を適用
                nn.Sigmoid()
            )    
        ])

    def forward(self, x):
        '''順伝播処理

        Parameter:
          x: 画像データまたは生成画像
        '''
        # 識別器のネットワークに入力して順伝播する
        for layer in self.layers:
            x = layer(x)
        
        # 出力されたテンソルの形状をフラット(bs,)にする
        return x.squeeze()

In [ ]:
'''
3. 生成器のクラスを定義
'''
import torch.nn as nn

class Generator(nn.Module):
    '''生成器のクラス

    Attributes:
      layers: Sequentialオブジェクトのリスト
    '''
    def __init__(self):
        '''生成器のネットワークを構築する
        '''
        super(Generator, self).__init__()

        input_dim = 100 # 入力データの次元
        out_ch = 128    # 最終層のチャネル数
        img_ch = 1      # 生成画像のチャネル数

        # 生成器のネットワークを定義する
        self.layers = nn.ModuleList([
            # 第1層: (bs, 100, 1, 1) -> (bs, 512, 3, 3)
            nn.Sequential(
                nn.ConvTranspose2d(input_dim,  # 入力のチャネル数は100
                                   out_ch * 4, # フィルター数は128×4
                                   3,          # 3×3のフィルター
                                   1,          # ストライドは1
                                   0),         # パディングは0(なし)
                # 出力値を正規化する(チャネル数は128×4)
                nn.BatchNorm2d(out_ch * 4),
                # ReLU関数を適用
                nn.ReLU()
            ),
            # 第2層: (bs, 512, 3, 3) -> (bs, 256, 7, 7)
            nn.Sequential(
                nn.ConvTranspose2d(out_ch * 4, # 入力のチャネル数は128×4
                                   out_ch * 2, # フィルター数は128×2
                                   3,          # 3×3のフィルター
                                   2,          # ストライドは2
                                   0),         # パディングは0(なし)
                # 出力値を正規化する(チャネル数は128×2)
                nn.BatchNorm2d(out_ch * 2),
                # ReLU関数を適用
                nn.ReLU()
            ),
            # 第3層: (bs, 256, 7, 7) -> (bs, 128, 14, 14)
            nn.Sequential(
                nn.ConvTranspose2d(out_ch * 2, # 入力のチャネル数は128×2
                                   out_ch,     # フィルター数は128
                                   4,          # 4×4のフィルター
                                   2,          # ストライドは2
                                   1), # 上下左右にサイズ1のパディング
                # 出力値を正規化する(チャネル数は128)
                nn.BatchNorm2d(out_ch),
                # ReLU関数を適用
                nn.ReLU()
            ),
            # 第4層: (bs, 128, 14, 14) -> (bs, 1, 28, 28)
            nn.Sequential(
                nn.ConvTranspose2d(out_ch, # 入力のチャネル数は128
                                   img_ch, # フィルター数は1
                                   4,      # 4×4のフィルター
                                   2,      # ストライドは2
                                   1), # 上下左右にサイズ1のパディング
                # Tanh関数を適用
                nn.Tanh()
            )
        ])

    def forward(self, z):
        '''順伝播処理

        Parameter:
          z: 識別器の出力
        '''
        # 生成器のネットワークに入力して順伝播する
        for layer in self.layers:
            z = layer(z)
        return z

In [ ]:
'''
4. 重みの初期化を行う関数
'''
def weights_init(m):
    '''
    DCGANの論文では重みを正規分布からサンプリングした値で初期化している
    
    Parameters:
      m: ネットワークのインスタンス
    '''
    classname = m.__class__.__name__
    # 畳み込み層の重み
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02) # 平均0、標準偏差0.02の正規分布
        m.bias.data.fill_(0) # バイアスのみ0で初期化
    # バッチ正規化層の重み
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02) # 平均1.0、標準偏差0.02の正規分布
        m.bias.data.fill_(0) # バイアスのみ0で初期化

In [ ]:
'''
5. 生成器をインスタンス化して重みを初期化する
'''
import torchsummary

# 生成器Generator
generator = Generator().to(device)
# 重みを初期化
generator.apply(weights_init)
# 生成器のサマリを出力
torchsummary.summary(generator,
                     (100, 1, 1))  # 入力テンソルの形状

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 3, 3]         461,312
       BatchNorm2d-2            [-1, 512, 3, 3]           1,024
              ReLU-3            [-1, 512, 3, 3]               0
   ConvTranspose2d-4            [-1, 256, 7, 7]       1,179,904
       BatchNorm2d-5            [-1, 256, 7, 7]             512
              ReLU-6            [-1, 256, 7, 7]               0
   ConvTranspose2d-7          [-1, 128, 14, 14]         524,416
       BatchNorm2d-8          [-1, 128, 14, 14]             256
              ReLU-9          [-1, 128, 14, 14]               0
  ConvTranspose2d-10            [-1, 1, 28, 28]           2,049
             Tanh-11            [-1, 1, 28, 28]               0
Total params: 2,169,473
Trainable params: 2,169,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

In [ ]:
'''
6. 識別器をインスタンス化して重みを初期化する
'''
# 識別器Discriminator
discriminator = Discriminator().to(device)
# 重みの初期化
discriminator.apply(weights_init)
# 識別器のサマリを出力
torchsummary.summary(discriminator,
                     (1, 28, 28))  # 入力テンソルの形状

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 14, 14]           2,176
         LeakyReLU-2          [-1, 128, 14, 14]               0
            Conv2d-3            [-1, 256, 7, 7]         524,544
       BatchNorm2d-4            [-1, 256, 7, 7]             512
         LeakyReLU-5            [-1, 256, 7, 7]               0
            Conv2d-6            [-1, 512, 3, 3]       1,180,160
       BatchNorm2d-7            [-1, 512, 3, 3]           1,024
         LeakyReLU-8            [-1, 512, 3, 3]               0
            Conv2d-9              [-1, 1, 1, 1]           4,609
          Sigmoid-10              [-1, 1, 1, 1]               0
Total params: 1,713,025
Trainable params: 1,713,025
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.78
Params size (MB): 6.53
Estimat

In [ ]:
'''
7. 損失関数とオプティマイザーの設定
'''
import torch.optim as optim

# 損失関数はバイナリクロスエントロピー誤差
criterion = nn.BCELoss()

# 識別器のオプティマイザ−を設定
optimizer_ds = optim.Adam(discriminator.parameters(),
                          # デフォルトの学習率0.001を論文で提案されている
                          # 0.0002に変更
                          lr=0.0002,
                          # 指数関数的減衰率としてデフォルトの(0.9, 0.999)
                          # のβ1の値のみ論文で提案されている(0.5, 0.999)に変更
                          betas=(0.5, 0.999)
                          )

# 生成器のオプティマイザーを設定
optimizer_gn = optim.Adam(generator.parameters(),
                          lr=0.0002,
                          betas=(0.5, 0.999)
                          )

In [ ]:
'''
8. エポックごとの画像生成に使用するノイズのテンソルを作成
'''
gn_input_dim = 100  # 生成器に入力するノイズの次元

# エポックごとに出力する生成画像のためのノイズを生成
# 標準正規分布からノイズを生成: 出力(bs, 100, 1, 1)
fixed_noise = torch.randn(
    batch_size,   # バッチサイズ
    gn_input_dim, # ノイズの次元100
    1,            # 1
    1,            # 1
    device=device)  

In [ ]:
%%time
'''
9. 学習を行う
'''
import torchvision.utils as vutils

# 学習回数
n_epoch = 10

# 画像の保存先のパス
# パスは環境に合わせて書き換えることが必要
outf = '/content/drive/MyDrive/Colab Notebooks/GAN/DCGAN_PyTorch/result'

# エポックごとに出力する生成画像のためのノイズを生成
fixed_noise = torch.randn(
    batch_size, gn_input_dim, 1, 1, device=device)  

# 学習のループ
for epoch in range(n_epoch):
    print('Epoch {}/{}'.format(epoch + 1, n_epoch))

    # バッチデータのループ(ステップ)
    for itr, data in enumerate(dataloader):
        # ミニバッチのすべての画像を取得
        real_image = data[0].to(device)
        # 画像の枚数を取得(バッチサイズ)
        sample_size = real_image.size(0)
        
        # 標準正規分布からノイズを生成: 出力(bs, 100, 1, 1)
        noise = torch.randn(sample_size, # バッチサイズ
                            gn_input_dim,# 生成器の入力次元100
                            1,           # 1
                            1,           # 1
                            device=device)
        # オリジナル画像に対する識別信号の正解値「1」で初期化した
        # (bs,)の形状のテンソルを生成
        real_target = torch.full((sample_size,),
                                 1.,
                                 device=device)
        # 生成画像に対する識別信号の正解値「0」で初期化した
        # (bs,)の形状のテンソルを生成
        fake_target = torch.full((sample_size,),
                                 0.,
                                 device=device) 
        
        # -----識別器の学習-----
        # 識別器の誤差の勾配を初期化
        discriminator.zero_grad()    

        # 識別器に画像を入力して識別信号を出力
        output = discriminator(real_image)
        # オリジナル画像に対する識別値の損失を取得
        ds_real_err = criterion(output,    # オリジナル画像の識別信号
                              real_target) # 正解ラベル(1)
        # 1ステップ(1バッチ)におけるオリジナル画像の識別信号の平均
        true_dsout_mean = output.mean().item()

        # ノイズを生成器に入力してフェイク画像を生成
        fake_image = generator(noise)
        # フェイク画像を識別器に入力して識別信号を出力
        output = discriminator(fake_image.detach())
        # フェイク画像を偽と判定できない場合の損失
        ds_fake_err = criterion(output,    # フェイク画像の識別信号
                              fake_target) # 正解ラベル(偽物の0)
        # フェイク画像の識別信号の平均
        fake_dsout_mean1 = output.mean().item()
        # オリジナル画像とフェイク画像に対する識別の損失を合計して
        # 識別器としての損失を求める
        ds_err = ds_real_err + ds_fake_err

        # 識別器全体の誤差を逆伝播
        ds_err.backward()
        # 判別器の重みのみを更新(生成器は更新しない)
        optimizer_ds.step()

        # -----生成器の学習-----
        # 生成器の誤差の勾配を初期化
        generator.zero_grad()
        # 更新した識別器に再度フェイク画像を入力して識別信号を取得
        output = discriminator(fake_image)
        # フェイク画像をオリジナル画像と誤認できない場合の損失
        gn_err = criterion(output,      # フェイク画像の識別信号
                           real_target) # 誤認させるのが目的なので正解ラベルは1
        # 更新後の識別器の誤差を逆伝播
        gn_err.backward() 
        # 更新後の識別器のフェイク画像に対する識別信号の平均
        fake_dsout_mean2 = output.mean().item()
        # 生成器の重みを更新後の識別誤差の勾配で更新
        optimizer_gn.step()

        # 100ステップごとに結果を出力
        if itr % 100 == 0: 
            print(
'({}/{}) ds_loss: {:.3f} - gn_loss: {:.3f} - true_out: {:.3f} - fake_out: {:.3f}>>{:.3f}'
                  .format(
                      itr + 1,          # ステップ数(イテレート回数)
                      len(dataloader),  # ステップ数(1エポックのバッチ数)
                      ds_err.item(),    # 識別器の損失
                      gn_err.item(),    # フェイクをオリジナルと誤認しない損失
                      true_dsout_mean,  # オリジナル画像の識別信号の平均
                      fake_dsout_mean1, # フェイク画像の識別信号の平均
                      fake_dsout_mean2) # 更新後識別器のフェイクの識別信号平均
                  )

        # 学習開始直後にオリジナル画像を保存する
        if epoch == 0 and itr == 0:
            vutils.save_image(real_image,
                              '{}/real_samples.png'.format(outf),
                              normalize=True,
                              nrow=10)

    # 1エポック終了ごとに生成器が生成した画像を保存
    # バッチサイズと同じ数のノイズを生成器に入力
    fake_image = generator(fixed_noise)
    # 画像を保存
    vutils.save_image(
        fake_image.detach(),
        '{}/generated_epoch_{:03d}.png'.format(outf, epoch + 1),
        normalize=True,
        nrow=10)

Epoch 1/10
(1/1200) ds_loss: 1.900 - gn_loss: 2.494 - true_out: 0.363 - fake_out: 0.487>>0.100
(101/1200) ds_loss: 0.244 - gn_loss: 6.108 - true_out: 0.924 - fake_out: 0.134>>0.004
(201/1200) ds_loss: 0.353 - gn_loss: 5.501 - true_out: 0.934 - fake_out: 0.224>>0.006
(301/1200) ds_loss: 0.261 - gn_loss: 4.070 - true_out: 0.820 - fake_out: 0.046>>0.028
(401/1200) ds_loss: 0.864 - gn_loss: 1.322 - true_out: 0.491 - fake_out: 0.009>>0.326
(501/1200) ds_loss: 0.443 - gn_loss: 1.579 - true_out: 0.730 - fake_out: 0.062>>0.281
(601/1200) ds_loss: 0.304 - gn_loss: 3.443 - true_out: 0.907 - fake_out: 0.165>>0.042
(701/1200) ds_loss: 0.374 - gn_loss: 2.955 - true_out: 0.894 - fake_out: 0.198>>0.063
(801/1200) ds_loss: 0.409 - gn_loss: 2.616 - true_out: 0.835 - fake_out: 0.166>>0.092
(901/1200) ds_loss: 0.274 - gn_loss: 3.750 - true_out: 0.948 - fake_out: 0.185>>0.029
(1001/1200) ds_loss: 0.304 - gn_loss: 2.212 - true_out: 0.808 - fake_out: 0.066>>0.142
(1101/1200) ds_loss: 0.480 - gn_loss: 2.334 